In [1]:
from input_preprocess import tokenizer
from lwm_model import lwm
import torch
import numpy as np

# 选择数据集
scenario_names = np.array([
    "city_18_denver", "city_15_indianapolis", "city_19_oklahoma",
    "city_12_fortworth", "city_11_santaclara", "city_7_sandiego"
])
scenario_idxs = np.array([0, 1, 2, 3, 4, 5])  # Select the scenario indexes
selected_scenario_names = scenario_names[scenario_idxs]

In [2]:
# 对数据进行标记
preprocessed_chs = tokenizer(
    selected_scenario_names=selected_scenario_names,  # Selects predefined DeepMIMOv3 scenarios_test. Set to None to load your own dataset.
    manual_data=None,  # If using a custom dataset, ensure it is a wireless channel dataset of size (N,32,32) based on the settings provided above.
    gen_raw=True  # Set gen_raw=False to apply masked channel modeling (MCM), as used in LWM pre-training. For inference, masking is unnecessary unless you want to evaluate LWM's ability to handle noisy inputs.
)


Basestation 3

UE-BS Channels
/Users/dianhongyang/Desktop/毕业设计/LWM_MODEL/LWM/scenarios/city_18_denver/BS3_UE_0-6970.mat


Generating channels: 100%|██████████| 6970/6970 [00:00<00:00, 41446.69it/s]



Basestation 3

UE-BS Channels
/Users/dianhongyang/Desktop/毕业设计/LWM_MODEL/LWM/scenarios/city_15_indianapolis/BS3_UE_0-6320.mat


Generating channels: 100%|██████████| 6320/6320 [00:00<00:00, 13884.68it/s]



Basestation 1

UE-BS Channels
/Users/dianhongyang/Desktop/毕业设计/LWM_MODEL/LWM/scenarios/city_19_oklahoma/BS1_UE_0-6150.mat


...

Generating channels: 100%|██████████| 6192/6192 [00:00<00:00, 21329.56it/s]



Basestation 1

UE-BS Channels
/Users/dianhongyang/Desktop/毕业设计/LWM_MODEL/LWM/scenarios/city_11_santaclara/BS1_UE_0-5358.mat


Generating channels: 100%|██████████| 5358/5358 [00:00<00:00, 15308.05it/s]



Basestation 1

UE-BS Channels
/Users/dianhongyang/Desktop/毕业设计/LWM_MODEL/LWM/scenarios/city_7_sandiego/BS1_UE_0-5893.mat


Processing items: 100%|██████████| 14840/14840 [00:00<00:00, 29245.59it/s]


In [3]:
# 加载模型
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Loading the LWM model on {device}...")
model = lwm.from_pretrained(device=device)

Loading the LWM model on cpu...
Model loaded successfully from /Users/dianhongyang/Desktop/毕业设计/LWM_Model/LWM/model_weights.pth to cpu


/Users/dianhongyang/Desktop/毕业设计/LWM_Model/LWM/lwm_model.py:123: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_path, map_location=devic

In [4]:
# 进行推理
from inference import lwm_inference, create_raw_dataset
input_types = ['cls_emb', 'channel_emb', 'raw']
selected_input_type = input_types[0]  # Change the index to select LWM CLS embeddings, LWM channel embeddings, or the original input channels.

if selected_input_type in ['cls_emb', 'channel_emb']:
    dataset = lwm_inference(preprocessed_chs, selected_input_type, model, device)
else:
    dataset = create_raw_dataset(preprocessed_chs, device)

LWM loss: 0.0193
